In [3]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn


In [164]:
import torch
import torch.nn as nn
import torch.optim as optim

class ModelSharpe(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ModelSharpe, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def _sharpe_loss(self, portfolio_returns):
        mean_returns = portfolio_returns.mean(dim=1)
        std_returns = portfolio_returns.std(dim=1)
        sharpe_ratio = mean_returns / std_returns
        return -sharpe_ratio.mean()

    def forward(self, x, y):
        output_lstm, (hn, cn) = self.lstm(x)
        unnormalized_weights = self.linear(output_lstm)
        weights = torch.softmax(unnormalized_weights, dim=-1)
        portfolio_returns = torch.sum(weights[:, :, :] * y[:, :, :], dim=2)
        loss = self._sharpe_loss(portfolio_returns)
        return loss

    def get_allocations(self, x):
        with torch.no_grad():
            output_lstm, (hn, cn) = self.lstm(x)
            unnormalized_weights = self.linear(output_lstm)
            weights = torch.softmax(unnormalized_weights, dim=-1)
        return weights[:, -1, :]


In [133]:
random_tensor = 2 * torch.rand(1, 50, 5) - 1
y = 2 * torch.rand(1, 50, 5) - 1

In [134]:
model_lstm = ModelSharpe(5, 64, 5)

In [135]:
A = model_lstm(random_tensor, y)
print(A.shape, A)
print(model_lstm.get_allocations(random_tensor))

torch.Size([]) tensor(0.1069, grad_fn=<NegBackward0>)
tensor([[0.1956, 0.2017, 0.1855, 0.2176, 0.1995]])


In [118]:
print(A[0].shape, A[1].shape, (A[0]*A[1]).shape, (torch.sum(A[0]*A[1])).shape)

torch.Size([1, 50, 5]) torch.Size([1, 50, 5]) torch.Size([1, 50, 5]) torch.Size([])


In [156]:
import numpy as np
import yfinance as yf
import pandas as pd

# Télécharger les données de 5 actifs (ici VTI, AGG, DBC, ^VIX pour 4 actifs)
tickers = ['VTI', 'AGG', 'DBC', '^VIX']
data = yf.download(tickers, start="2006-01-01", end="2020-12-31", interval="1d")['Adj Close']
data_na = data.dropna(axis = 0)
for column in data_na.columns:
    data_na[f'{column}_R'] = data_na[f'{column}'].pct_change()
    data_na[f'{column}_y'] = data_na[f'{column}'].shift(-1)
data_na.dropna(axis=0, inplace=True)
data_na

[*********************100%***********************]  4 of 4 completed
/var/folders/7q/v0vj3h051dn717mx7ngrr8yh0000gn/T/ipykernel_65029/1044928713.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_na[f'{column}_R'] = data_na[f'{column}'].pct_change()
/var/folders/7q/v0vj3h051dn717mx7ngrr8yh0000gn/T/ipykernel_65029/1044928713.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_na[f'{column}_y'] = data_na[f'{column}'].shift(-1)
/var/folders/7q/v0vj3h051dn717mx7ngrr8yh0000gn/T/ipykernel_65029/10449

Ticker,AGG,DBC,VTI,^VIX,AGG_R,AGG_y,DBC_R,DBC_y,VTI_R,VTI_y,^VIX_R,^VIX_y
Date,,,,,,,,,,,,
2006-02-07 00:00:00+00:00,56.262386,20.285254,44.219505,13.590000,-0.000699,56.234272,-0.028926,20.198935,-0.009736,44.537621,0.042178,12.830000
2006-02-08 00:00:00+00:00,56.234272,20.198935,44.537621,12.830000,-0.000500,56.267994,-0.004255,20.388840,0.007194,44.452770,-0.055923,13.120000
2006-02-09 00:00:00+00:00,56.267994,20.388840,44.452770,13.120000,0.000600,56.149998,0.009402,20.017662,-0.001905,44.544682,0.022603,12.870000
2006-02-10 00:00:00+00:00,56.149998,20.017662,44.544682,12.870000,-0.002097,56.189373,-0.018205,19.706909,0.002068,44.343208,-0.019055,13.350000
2006-02-13 00:00:00+00:00,56.189373,19.706909,44.343208,13.350000,0.000701,56.099419,-0.015524,19.542898,-0.004523,44.763840,0.037296,12.250000
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-22 00:00:00+00:00,106.425880,13.624198,181.857071,24.230000,0.001443,106.353745,-0.011004,13.794738,0.000207,182.168060,-0.036963,23.309999
2020-12-23 00:00:00+00:00,106.353745,13.794738,182.168060,23.309999,-0.000678,106.452980,0.012517,13.832635,0.001710,182.472809,-0.037969,21.530001
2020-12-24 00:00:00+00:00,106.452980,13.832635,182.472809,21.530001,0.000933,106.471016,0.002747,13.747366,0.001673,183.627289,-0.076362,21.700001


In [171]:
def create_batches(dataframe, start_date, end_date, window_size=50):
    filtered_data = dataframe.loc[start_date:end_date]
    
    x_batches = []
    y_batches = []

    for i in range(len(filtered_data) - window_size):
        window_x = filtered_data.iloc[i:i+window_size][[f'{col}_R' for col in data.columns]].values
        window_y = filtered_data.iloc[i:i+window_size][[f'{col}_y' for col in data.columns]].values

        x_batches.append(window_x)
        y_batches.append(window_y)

    return np.array(x_batches), np.array(y_batches)

x_batches, y_batches = create_batches(data_na, start_date="2006-01-01", end_date="2010-12-31")


KeyError: "None of [Index(['AGG_y_y', 'DBC_y_y', 'VTI_y_y', '^VIX_y_y'], dtype='object', name='Ticker')] are in the [columns]"

In [172]:
def create_batches(dataframe, start_date, end_date, window_size=50):
    filtered_data = dataframe.loc[start_date:end_date]
    
    x_batches = []
    y_batches = []

    for i in range(len(filtered_data) - window_size):
        # Sélection correcte des colonnes de rendements et de prix
        window_returns = filtered_data.iloc[i:i+window_size][[f'{col}_R' for col in dataframe.columns]].values
        window_prices = filtered_data.iloc[i:i+window_size][[col for col in dataframe.columns]].values
        
        # Combinaison des rendements et des prix
        window_x = np.concatenate([window_returns, window_prices], axis=1)
        window_y = filtered_data.iloc[i:i+window_size][[f'{col}_y' for col in dataframe.columns if '_y' in col]].values

        x_batches.append(window_x)
        y_batches.append(window_y)

    return np.array(x_batches), np.array(y_batches)

x_batches, y_batches = create_batches(data_na, start_date="2006-01-01", end_date="2010-12-31")


KeyError: "['AGG_R_R', 'AGG_y_R', 'DBC_R_R', 'DBC_y_R', 'VTI_R_R', 'VTI_y_R', '^VIX_R_R', '^VIX_y_R'] not in index"

In [173]:
data_na

Ticker,AGG,DBC,VTI,^VIX,AGG_R,AGG_y,DBC_R,DBC_y,VTI_R,VTI_y,^VIX_R,^VIX_y
Date,,,,,,,,,,,,
2006-02-07 00:00:00+00:00,56.262386,20.285254,44.219505,13.590000,-0.000699,56.234272,-0.028926,20.198935,-0.009736,44.537621,0.042178,12.830000
2006-02-08 00:00:00+00:00,56.234272,20.198935,44.537621,12.830000,-0.000500,56.267994,-0.004255,20.388840,0.007194,44.452770,-0.055923,13.120000
2006-02-09 00:00:00+00:00,56.267994,20.388840,44.452770,13.120000,0.000600,56.149998,0.009402,20.017662,-0.001905,44.544682,0.022603,12.870000
2006-02-10 00:00:00+00:00,56.149998,20.017662,44.544682,12.870000,-0.002097,56.189373,-0.018205,19.706909,0.002068,44.343208,-0.019055,13.350000
2006-02-13 00:00:00+00:00,56.189373,19.706909,44.343208,13.350000,0.000701,56.099419,-0.015524,19.542898,-0.004523,44.763840,0.037296,12.250000
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-22 00:00:00+00:00,106.425880,13.624198,181.857071,24.230000,0.001443,106.353745,-0.011004,13.794738,0.000207,182.168060,-0.036963,23.309999
2020-12-23 00:00:00+00:00,106.353745,13.794738,182.168060,23.309999,-0.000678,106.452980,0.012517,13.832635,0.001710,182.472809,-0.037969,21.530001
2020-12-24 00:00:00+00:00,106.452980,13.832635,182.472809,21.530001,0.000933,106.471016,0.002747,13.747366,0.001673,183.627289,-0.076362,21.700001


In [162]:
print(x_batches.shape, y_batches.shape)

(1185, 50, 4) (1185, 50, 4)


In [168]:
mini_batch_size = 10

x_batches_tensor = torch.tensor(x_batches, dtype=torch.float32)
y_batches_tensor = torch.tensor(y_batches, dtype=torch.float32)

num_batches = x_batches_tensor.shape[0]

input_size = 8
hidden_size = 64
output_size = 4

model = ModelSharpe(input_size, hidden_size, output_size)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    
    for i in range(0, num_batches, mini_batch_size):
        x_mini_batch = x_batches_tensor[i:i+mini_batch_size]
        y_mini_batch = y_batches_tensor[i:i+mini_batch_size]
        
        optimizer.zero_grad()
        
        loss = model(x_mini_batch, y_mini_batch)
        
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

Epoch [1/100], Loss: -147.432861328125
Epoch [2/100], Loss: -188.94393920898438
Epoch [3/100], Loss: -200.84300231933594
Epoch [4/100], Loss: -140.29714965820312
Epoch [5/100], Loss: -172.34378051757812
Epoch [6/100], Loss: -145.18955993652344
Epoch [7/100], Loss: -149.67581176757812
Epoch [8/100], Loss: -151.96597290039062
Epoch [9/100], Loss: -143.82693481445312
Epoch [10/100], Loss: -144.73297119140625
Epoch [11/100], Loss: -137.72666931152344
Epoch [12/100], Loss: -124.92757415771484
Epoch [13/100], Loss: -124.01005554199219
Epoch [14/100], Loss: -119.18391418457031
Epoch [15/100], Loss: -114.2684326171875
Epoch [16/100], Loss: -110.49337005615234
Epoch [17/100], Loss: -122.0008316040039
Epoch [18/100], Loss: -104.09810638427734
Epoch [19/100], Loss: -147.58187866210938
Epoch [20/100], Loss: -102.47021484375
Epoch [21/100], Loss: -127.6596908569336
Epoch [22/100], Loss: -147.04251098632812
Epoch [23/100], Loss: -137.7582550048828
Epoch [24/100], Loss: -122.93885803222656
Epoch [25/